# Debug training
May 19, 2021

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

import socket

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
base_dict={'cori':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/',
         'summit':'/autofs/nccs-svm1_home1/venkitesh/projects/cosmogan/cosmogan_pytorch/'}
facility='cori' if socket.gethostname()[:4]=='cori' else 'summit'

base_dir=base_dict[facility]

In [5]:
sys.path.append(base_dir+'/code/modules_image_analysis/')
from modules_img_analysis import *
# sys.path.append(base_dir+'/code/5_3d_cgan/1_main_code/')
# import post_analysis_pandas as post

In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
dict1={'cori':{
'2d':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128_square/',
'3d':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3D/',
'3d_cgan':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'},
'summit':{'2d':'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/2d/',
          '3d':'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/'}}

In [9]:
# parent_dir=u.result
parent_dir=dict1[facility]['3d']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20210522*')]
dir_lst
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210522_132418_cgan_bs16_nodes8_lrg0.001_lrd0.0008-…

In [10]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/20210522_132418_cgan_bs16_nodes8_lrg0.001_lrd0.0008-fastvary


## Plot Losses

In [11]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)
# df_metrics.tail(10)
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)

f_plot_metrics(df_metrics,['hist_chi'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
f_plot_metrics(df_metrics,['lr_d','lr_g'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [14]:
df_metrics[(df_metrics.lr_d>=6.69e-04) ]

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
0,0.0,0.0,0.684328,0.744878,1.429206,14.985600,26.546993,11.561393,1.649161,NaN,NaN,NaN,NaN,0.031475,0.102772,-14.985594,14.688859,0.0008,0.001
1,1.0,0.0,0.336497,5.654778,5.991275,23.825830,34.785828,10.959996,1.042977,inf,1.078888,NaN,NaN,4.616751,6.047825,-23.825830,0.452803,0.0008,0.001
2,2.0,0.0,19.139462,1.121398,20.260859,8.199556,inf,inf,0.875772,inf,1.073260,NaN,NaN,-20.943382,-21.892569,-8.198584,0.656637,0.0008,0.001
3,3.0,0.0,7.172502,0.407760,7.580261,0.003163,inf,inf,1.003720,inf,1.059141,NaN,NaN,-7.679461,-7.321924,7.211850,0.422542,0.0008,0.001
4,4.0,0.0,0.225813,6.585375,6.811188,0.083722,inf,inf,0.944587,inf,1.002694,NaN,NaN,7.258518,7.137567,2.589293,0.456555,0.0008,0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,525.0,9.0,0.241629,0.233197,0.474826,10.148956,21.414413,11.265456,1.253028,10.962863,1.022404,NaN,NaN,2.510684,-1.878741,-10.148359,0.437776,0.0008,0.001
526,526.0,9.0,2.776093,0.702280,3.478373,7.032230,18.239979,11.207748,1.450263,10.987443,1.106873,NaN,NaN,-2.978030,-9.320860,-7.025029,0.454145,0.0008,0.001
527,527.0,9.0,0.516433,0.358227,0.874660,0.477100,11.842767,11.365666,1.751478,11.012884,1.530485,NaN,NaN,0.614733,-5.732837,0.874437,0.472343,0.0008,0.001
528,528.0,9.0,0.557273,2.949306,3.506579,0.631434,11.958301,11.326866,1.989422,11.040251,1.789453,NaN,NaN,4.975990,2.880639,0.409925,0.442258,0.0008,0.001


In [20]:
df_metrics.plot(kind='scatter',x='step',y=['lr_d'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='step', ylabel='[lr_d]'>

In [16]:
np.unique(df_metrics.lr_d.values),np.unique(df_metrics.lr_g.values)

# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

(array([5.e-05, 1.e-04, 2.e-04, 4.e-04, 8.e-04,    nan]),
 array([0.001,   nan]))

## Calculating learn rates

In [17]:
Nsteps=5;Lf=0.00005;Li=0.0008
Lambda=(Lf/Li)**(1.0/Nsteps)
print(Lambda,Lambda**2)

lst=[10,40,60,70,80,100]
# lst=range(1,11)
[(Li*Lambda**(count+1),i) for count,i in enumerate(lst)]

0.5743491774985174 0.3298769776932235


[(0.000459479341998814, 10),
 (0.0002639015821545788, 40),
 (0.00015157165665103975, 60),
 (8.705505632961239e-05, 70),
 (4.9999999999999975e-05, 80),
 (2.8717458874925857e-05, 100)]

In [19]:
Lambda=0.25;Li=0.004
lst=[10,40,60,70,80,100,140,180]
# lst=range(1,11)
[(Li*Lambda**(count+1),i) for count,i in enumerate(lst)]

[(0.001, 10),
 (0.00025, 40),
 (6.25e-05, 60),
 (1.5625e-05, 70),
 (3.90625e-06, 80),
 (9.765625e-07, 100),
 (2.44140625e-07, 140),
 (6.103515625e-08, 180)]

## Grid plot of images

In [46]:
epoch=170
flist=glob.glob(result_dir+'/images/gen_img*_epoch-{0}_step*'.format(epoch))
steps_list=[fname.split('/')[-1].split('step-')[-1].split('.')[0] for fname in flist]

print(*steps_list)

9530 9540 9540 9540 9550 9530 9570 9570 9530 9560 9560 9550 9570 9550 9560


In [47]:
# fname=flist[0]
# fname,fname.split('/')[-1].split('step-')[-1].split('.')[0]

step=9550
fname=glob.glob(result_dir+'/images/gen_img_*_epoch-{0}_step-{1}.npy'.format(epoch,step))[0]
fname

'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/20210519_81818_cgan_bs16_lr0.001_nodes8_spec0.1/images/gen_img_label-0.5_epoch-170_step-9550.npy'

In [48]:
images=np.load(fname)[:,0,:,:]
print(images.shape)
f_plot_grid(images[:8,:,:,0],cols=4,fig_size=(8,4))

(32, 64, 64, 64)
2 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …